In [ ]:

import os
import gurobipy
import cobra
from cobra.io import read_sbml_model, write_sbml_model,load_matlab_model, save_matlab_model
import itertools as it
import numpy as np
import pandas as pd
import re
import csv
import json
import requests

In [ ]:
os.environ['GUROBI_HOME'] = "/home/jiang817/cybergut_project/Software/gurobi_license"
os.environ['GRB_LICENSE_FILE'] = "/home/jiang817/cybergut_project/Software/gurobi_license/gurobi.lic"

In [ ]:
# load draft reconstruction and Recon3D
General_model = read_sbml_model('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Draft_reconstructions/Recon3D.xml')
model = read_sbml_model('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/Draft_reconstructions/Draft_reconstruction_consensus/consensus_recon.xml')


In [ ]:
# load the component of DMEM
medium_df= pd.read_excel('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Cybergut/metabolic_task/metabolic_task_testing_input/DMEM inputs.xlsx', header=0)
DMEM = medium_df.iloc[:,0].to_list()


In [ ]:
# Define metabolic task testing function
def test_metabolic_task(model, General_model, tasks, Medium):

# model: draft reconstruction
# tasks: daraframe
# General_model: models that draft reconstructions came from
# Medium: boolen--True: using DMEM medium; False: not using DMEM medium  
    
    # close exchange reactions
    for i in range (len(model.boundary)):
        if model.boundary[i].id.startswith('EX_'):
            model.boundary[i].bounds = (0, 100)

    #  close all sinck reactions
    for i in range (len(model.boundary)):
        if model.boundary[i].id.startswith('SK_'):
            model.boundary[i].bounds = (0, 0)

   # close demand reactions
    for i in range (len(model.boundary)):
        if model.boundary[i].id.startswith('DM_'):
            model.boundary[i].lower_bound = 0

    # aerobic
    if 'EX_o2_e' not in [model.reactions[i].id for i in range(len(model.reactions))]:
        model.add_reactions([General_model.reactions.get_by_id('EX_o2_e')])
    model.reactions.get_by_id('EX_o2_e').bounds = (-40, 40)

    # change all reaction coefficient to 0
    Cofficient = {}
    for i in model.reactions:
        Cofficient[i] = 0
    model.objective = Cofficient

    # using Ham medium
    if Medium:
        model_boundary = [model.boundary[i].id for i in range(len(model.boundary))]
        for c in DMEM:
            if c in model_boundary:
                model.reactions.get_by_id(c).lower_bound = -1
            else:
                c_met = c.split('EX_')[1]
                print(c_met + ": Ham component not in draft reconstrcution")
                model.add_boundary(General_model.metabolites.get_by_id(c_met), type="exchange")
                model.reactions.get_by_id(c).bounds = (-1, 100)
    
    tol = 1e-6
    
    # outputs
    TestSolution = np.array([])

    missed_mets = {}
    
    
    # testing
    mdoel_mets = [model.metabolites[i].id for i in range(len(model.metabolites))]

    for i in range(len(tasks)):
        modelOri_copy = model.copy()
        # removed objective 
        Cofficient = {}
        for n in modelOri_copy.reactions:
            Cofficient[n] = 0
        modelOri_copy.objective = Cofficient
      
        task_all_info = tasks.iloc[i].to_list()
        task_name = task_all_info[0]
        reactants = task_all_info[1]
        #reactantsLB = task_all_info[2]
        product = task_all_info[3]
        #productLB = task_all_info[4]
        missed_mets[task_name] = []

       ## Set sink reactions for reactants
        reactants = reactants.split(',')
        reactantsLB = reactantsLB.split(',')
        
        for i in range(len(reactants)):
            reactant = reactants[i]
            Reactant_LB = -float(reactantsLB[i])
            if reactant not in mdoel_mets:
                print(reactant, 'not included in the model')
                missed_mets[task_name].append(reactant)
                modelOri_copy.add_metabolites(General_model.metabolites.get_by_id(reactant))
            # add the above metabolties to sink reactions
            # set the boundary of consumed metabolite sink rxn with (-1, -1)
            # set the boundary of produced metabolite sink rxn with (0, 100)  
            reactant_SK = "SK_" + reactant
            try:
                modelOri_copy.add_boundary(modelOri_copy.metabolites.get_by_id(reactant), type="sink")
            except:
                print(reactant_SK, 'is already in the model')
            modelOri_copy.reactions.get_by_id(reactant_SK).bounds = (-1, 1000)
                
        ## Set sink reactions for product
        if product not in mdoel_mets:
            print(product, 'not included in the model')
            missed_mets[task_name].append(product)
            modelOri_copy.add_metabolites(General_model.metabolites.get_by_id(product))
        product_SK = "SK_" + product
        try:
            modelOri_copy.add_boundary(modelOri_copy.metabolites.get_by_id(product), type="sink")
        except:
            print(product_SK, 'is already in the model')
        modelOri_copy.reactions.get_by_id(product_SK).bounds = (0, 100)


        # set the produced metabolite sink rxn as objective and then optimize the model
        modelOri_copy.objective = product_SK
        print(modelOri_copy.objective.expression)
        print(modelOri_copy.objective.direction)
        FBA = modelOri_copy.optimize('maximize')
        print(FBA.objective_value)
        TestSolution = np.append(TestSolution, FBA.objective_value)
        print(len(TestSolution))
        #print(TestSolution)
        print('----------------')

    tasks['results'] = TestSolution
   
    missed_met_df = pd.DataFrame.from_dict(missed_mets, orient='index')

    return tasks, missed_met_df

In [ ]:
# load metabolic tasks_1
tasks_1 = pd.read_excel('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Neuron_GEM/Metabolic_tasks/Metabolic_tasks_1.xlsx')
# run the metabolic task testing without DMEM
Result1, miss_met = test_metabolic_task(model, General_model, tasks_1, False)


In [ ]:
# load metabolic tasks_2
tasks_2 = pd.read_excel('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Neuron_GEM/Metabolic_tasks/Metabolic_tasks_2.xlsx')
# run the metabolic task testing without DMEM
Result2, miss_met = test_metabolic_task(model, General_model, tasks_2, True)